In [3]:
from sklearn import preprocessing

SETSofVALS = 25
def generateHeader():
    header = []
    for i in range(1,SETSofVALS+1):
        header.append("x"+str(i))
        header.append("y"+str(i))
        header.append("z"+str(i))
        header.append("block"+str(i))
    header.append("biome")
    return header

def encode_columns_dummy(df, header):
    for i in range(1, SETSofVALS+1):
        encode_text_dummy(df,header+str(i))
        
#code below taken from https://github.com/jeffheaton/t81_558_deep_learning
# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)
    
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


In [13]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint


path = "./Data/"

filename_read = os.path.join(path,"data_cleaned2.csv")
#header = generateHeader()
df = pd.read_csv(filename_read)

#shuffling - not required as the splitting into train/test does the shuffling for us
#df = df.reindex(np.random.permutation(df.index))
#df.reset_index(inplace=True, drop=True)

biomes = encode_text_index(df,"biome")
encode_columns_dummy(df, "block")
x,y = to_xy(df,"biome")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25, random_state=42)

model = Sequential()
model.add(Dense(100, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(50, activation='relu')) # Hidden 2
model.add(Dense(25, activation='relu')) # Hidden 3
model.add(Dense(y.shape[1],activation='softmax')) # Output
model.compile(loss='categorical_crossentropy', optimizer='adam')
#early stopping based on 
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=0, save_best_only=True) # save best model

model.fit(x,y,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=2,epochs=1000)
model.load_weights('best_weights.hdf5') # load weights from best model

df

/Users/saadghani/Development/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Train on 6623 samples, validate on 1656 samples
Epoch 1/1000
 - 1s - loss: 1.5793 - val_loss: 1.3297
Epoch 2/1000
 - 0s - loss: 1.2956 - val_loss: 1.1892
Epoch 3/1000
 - 0s - loss: 1.2122 - val_loss: 1.1024
Epoch 4/1000
 - 0s - loss: 1.1600 - val_loss: 1.0643
Epoch 5/1000
 - 0s - loss: 1.1079 - val_loss: 1.0559
Epoch 6/1000
 - 0s - loss: 1.0732 - val_loss: 1.0782
Epoch 7/1000
 - 1s - loss: 1.0785 - val_loss: 1.0723
Epoch 8/1000
 - 0s - loss: 1.0036 - val_loss: 0.9387
Epoch 9/1000
 - 0s - loss: 1.0009 - val_loss: 0.9470
Epoch 10/1000
 - 1s - loss: 0.9402 - val_loss: 0.9276
Epoch 11/1000
 - 0s - loss: 0.9618 - val_loss: 0.8767
Epoch 12/1000
 - 1s - loss: 0.9111 - val_loss: 0.9324
Epoch 13/1000
 - 0s - loss: 0.8890 - val_loss: 0.8858
Epoch 14/1000
 - 1s - loss: 0.8955 - val_loss: 0.9248
Epoch 15/1000
 - 1s - loss: 0.8618 - val_loss: 0.7904
Epoch 16/1000
 - 1s - loss: 0.8534 - val_loss: 0.8021
Epoch 17/1000
 - 0s - loss: 0.8697 - val_loss: 0.9404
Epoch 18/1000
 - 1s - loss: 0.8650 - val_lo

,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,...,block25-20518,block25-24585,block25-24587,block25-24614,block25-32786,block25-32806,block25-32874,block25-36882,block25-40978,block25-41135
0,68,68,71,69,69,68,68,69,69,69,...,0,0,0,0,0,0,0,0,0,0
1,69,69,69,70,70,69,69,69,70,70,...,0,0,0,0,0,0,0,0,0,0
2,68,69,69,69,69,69,69,69,69,69,...,0,0,0,0,0,0,0,0,0,0
3,68,68,68,68,68,68,68,68,68,68,...,0,0,0,0,1,0,0,0,0,0
4,69,67,67,68,68,69,68,68,67,68,...,0,0,0,0,0,0,0,0,0,0
5,69,66,68,68,68,69,66,67,67,68,...,0,0,0,0,0,0,0,0,0,0
6,71,67,73,67,68,71,67,68,68,68,...,0,0,0,0,0,0,0,0,0,0
7,68,68,69,69,69,68,68,69,69,69,...,0,0,0,0,0,0,0,0,0,0
8,69,69,70,70,70,69,69,70,70,70,...,0,0,0,0,0,0,0,0,0,0
9,70,70,70,70,70,70,70,70,70,71,...,0,0,0,0,0,0,0,0,0,0


In [14]:
pred = model.predict(x)
print("Shape: {}".format(pred.shape))
print(pred)

Shape: (6623, 5)
[[4.6159033e-02 5.3200072e-01 4.8937388e-02 2.7796444e-01 9.4938442e-02]
 [7.3361270e-02 5.3385365e-01 4.9719077e-02 2.6015779e-01 8.2908265e-02]
 [6.4905927e-02 5.4078311e-01 4.2336240e-02 2.7121279e-01 8.0761924e-02]
 ...
 [1.2017388e-11 1.9573738e-05 5.9332553e-15 1.2920626e-06 9.9997914e-01]
 [9.8109571e-04 6.7038886e-02 4.3683648e-04 4.2923321e-03 9.2725080e-01]
 [1.9527908e-08 4.4798497e-03 3.4736962e-09 2.0879002e-06 9.9551803e-01]]


In [15]:
predict_classes = np.argmax(pred,axis=1)
expected_classes = np.argmax(y,axis=1)
print(f"Predictions: {predict_classes}")
print(f"Expected: {expected_classes}")

Predictions: [1 1 1 ... 4 4 4]
Expected: [3 3 3 ... 4 4 4]


In [16]:
from sklearn.metrics import accuracy_score
# Accuracy might be a more easily understood error metric.  It is essentially a test score.  For all of the iris predictions,
# what percent were correct?  The downside is it does not consider how confident the neural network was in each prediction.
correct = accuracy_score(expected_classes,predict_classes)
print(f"Accuracy: {correct}")

Accuracy: 0.7055714932809904


In [ ]:
from IPython.display import display

# Don't display numpy in scientific notation
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

# Generate predictions
pred = model.predict(x_test)

print("Numpy array of predictions")
print(pred[0]*100)

print("As percent probability")
display(pred[0:5])

score = metrics.log_loss(y_test, pred)
print("Log loss score: {}".format(score))